<a href="https://colab.research.google.com/github/IbrahimAllMamun/flood_incidence/blob/main/flood_incidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from sklearn.preprocessing import QuantileTransformer


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
!wget https://raw.githubusercontent.com/IbrahimAllMamun/flood_incidence/main/flood.csv

--2025-09-14 08:10:52--  https://raw.githubusercontent.com/IbrahimAllMamun/flood_incidence/main/flood.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2071590 (2.0M) [text/plain]
Saving to: ‘flood.csv’

flood.csv           100%[===================>]   1.98M  --.-KB/s    in 0.07s   

2025-09-14 08:10:52 (27.3 MB/s) - ‘flood.csv’ saved [2071590/2071590]



In [3]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

In [17]:
df = pd.read_csv('flood.csv')
df.rename(columns={'Flood?':'Flood'}, inplace=True)
df = df.sort_values(["Year", "Month", "Station_Number"])
df.drop(columns=['Sl', 'LATITUDE', 'LONGITUDE', 'Period'], inplace=True)
df.fillna({'Flood':0}, inplace=True)
display(df.head())

,Station_Names,Year,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,ALT,Flood
5892,Dinajpur,1948,1,28.9,10.4,0.000000,60.00000,0.624074,1.3,5.572000,41863,365790.2,834718.0,37,0.0
1356,Bogra,1948,1,25.8,11.7,8.140625,74.21875,0.875926,1.1,7.296226,41883,435303.7,751187.5,20,0.0
13188,Mymensingh,1948,1,28.3,11.7,0.000000,67.00000,0.594444,1.7,6.134286,41886,540975.3,737535.8,19,0.0
18288,Srimangal,1948,1,27.7,9.0,8.015385,75.00000,0.464815,2.4,6.710714,41915,675761.6,687095.9,23,0.0
6684,Faridpur,1948,1,27.8,11.9,3.000000,91.00000,0.746296,2.9,6.986207,41929,483877.3,610719.3,9,0.0


In [15]:
# df = df[df['Station_Names'] != "Chittagong (City-Ambagan)"]
# display(df.head())
# print(df.shape)

,Station_Names,Year,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,ALT,Flood,Month_sin,Month_cos,Time
5892,Dinajpur,1948,1,28.9,10.4,0.000000,60.00000,0.624074,1.3,5.572000,41863,365790.2,834718.0,37,0.0,0.0,1.0,1948-01-01
1356,Bogra,1948,1,25.8,11.7,8.140625,74.21875,0.875926,1.1,7.296226,41883,435303.7,751187.5,20,0.0,0.0,1.0,1948-01-01
13188,Mymensingh,1948,1,28.3,11.7,0.000000,67.00000,0.594444,1.7,6.134286,41886,540975.3,737535.8,19,0.0,0.0,1.0,1948-01-01
18288,Srimangal,1948,1,27.7,9.0,8.015385,75.00000,0.464815,2.4,6.710714,41915,675761.6,687095.9,23,0.0,0.0,1.0,1948-01-01
6684,Faridpur,1948,1,27.8,11.9,3.000000,91.00000,0.746296,2.9,6.986207,41929,483877.3,610719.3,9,0.0,0.0,1.0,1948-01-01


(20496, 18)


In [18]:
df.shape

(20544, 15)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20544 entries, 5892 to 20543
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Station_Names      20544 non-null  object 
 1   Year               20544 non-null  int64  
 2   Month              20544 non-null  int64  
 3   Max_Temp           20544 non-null  float64
 4   Min_Temp           20544 non-null  float64
 5   Rainfall           20544 non-null  float64
 6   Relative_Humidity  20544 non-null  float64
 7   Wind_Speed         20544 non-null  float64
 8   Cloud_Coverage     20544 non-null  float64
 9   Bright_Sunshine    20544 non-null  float64
 10  Station_Number     20544 non-null  int64  
 11  X_COR              20544 non-null  float64
 12  Y_COR              20544 non-null  float64
 13  ALT                20544 non-null  int64  
 14  Flood              20544 non-null  float64
dtypes: float64(10), int64(4), object(1)
memory usage: 2.5+ MB


In [6]:
df["Month_sin"] = np.sin(2 * np.pi * (df["Month"] - 1) / 12.0)
df["Month_cos"] = np.cos(2 * np.pi * (df["Month"] - 1) / 12.0)

In [7]:
df['Time'] = pd.to_datetime(df[['Year', 'Month']].assign(Day=1))
display(df.head())

,Station_Names,Year,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,ALT,Flood,Month_sin,Month_cos,Time
5892,Dinajpur,1948,1,28.9,10.4,0.000000,60.00000,0.624074,1.3,5.572000,41863,365790.2,834718.0,37,0.0,0.0,1.0,1948-01-01
1356,Bogra,1948,1,25.8,11.7,8.140625,74.21875,0.875926,1.1,7.296226,41883,435303.7,751187.5,20,0.0,0.0,1.0,1948-01-01
13188,Mymensingh,1948,1,28.3,11.7,0.000000,67.00000,0.594444,1.7,6.134286,41886,540975.3,737535.8,19,0.0,0.0,1.0,1948-01-01
18288,Srimangal,1948,1,27.7,9.0,8.015385,75.00000,0.464815,2.4,6.710714,41915,675761.6,687095.9,23,0.0,0.0,1.0,1948-01-01
6684,Faridpur,1948,1,27.8,11.9,3.000000,91.00000,0.746296,2.9,6.986207,41929,483877.3,610719.3,9,0.0,0.0,1.0,1948-01-01


In [8]:
time_varying = [
    "Max_Temp", "Min_Temp", "Rainfall", "Relative_Humidity",
    "Wind_Speed", "Cloud_Coverage", "Bright_Sunshine",
    "Month_sin", "Month_cos"
]
static_feats = ["ALT", "X_COR", "Y_COR"]

In [9]:
def build_sequences_monthly(df, window=12):
    X_seq, X_static, y, groups = [], [], [], []

    for st, g in df.groupby("Station_Number"):
        g = g.sort_values(["Year", "Month"])

        seq_data = g[time_varying].to_numpy(dtype=float)
        static_vals = g[static_feats].iloc[0].to_numpy(dtype=float)
        labels = g["Flood"].to_numpy(dtype=int)
        years = g["Year"].to_numpy()
        months = g["Month"].to_numpy()

        for i in range(len(g) - window):
            X_seq.append(seq_data[i:i+window])
            X_static.append(static_vals)
            y.append(labels[i+window])  # predict flood at next month
            groups.append((st, years[i+window], months[i+window]))

    return (
        np.array(X_seq),
        np.array(X_static),
        np.array(y).astype(int),
        pd.DataFrame(groups, columns=["Station", "Year", "Month"])
    )

# Example
X_seq, X_static, y, groups = build_sequences_monthly(df, window=12)
print("Sequences:", X_seq.shape)   # (N, 12, n_features)
print("Static:", X_static.shape)   # (N, 3)
print("Labels:", y.shape)          # (N,)
print("Positives:", y.sum(), "Negatives:", (y==0).sum())


Sequences: (20148, 12, 9)
Static: (20148, 3)
Labels: (20148,)
Positives: 4048 Negatives: 16100


In [32]:
np.unique(groups['Year'])

array([1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959,
       1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013])

In [10]:
X_static

array([[3.400000e+01, 4.261729e+05, 8.448223e+05],
       [3.400000e+01, 4.261729e+05, 8.448223e+05],
       [3.400000e+01, 4.261729e+05, 8.448223e+05],
       ...,
       [4.000000e+00, 7.347654e+05, 3.089141e+05],
       [4.000000e+00, 7.347654e+05, 3.089141e+05],
       [4.000000e+00, 7.347654e+05, 3.089141e+05]])

In [19]:
df.head()

,Station_Names,Year,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,ALT,Flood
5892,Dinajpur,1948,1,28.9,10.4,0.000000,60.00000,0.624074,1.3,5.572000,41863,365790.2,834718.0,37,0.0
1356,Bogra,1948,1,25.8,11.7,8.140625,74.21875,0.875926,1.1,7.296226,41883,435303.7,751187.5,20,0.0
13188,Mymensingh,1948,1,28.3,11.7,0.000000,67.00000,0.594444,1.7,6.134286,41886,540975.3,737535.8,19,0.0
18288,Srimangal,1948,1,27.7,9.0,8.015385,75.00000,0.464815,2.4,6.710714,41915,675761.6,687095.9,23,0.0
6684,Faridpur,1948,1,27.8,11.9,3.000000,91.00000,0.746296,2.9,6.986207,41929,483877.3,610719.3,9,0.0


In [11]:
X_seq[1,:,:]

array([[ 2.97228070e+01,  1.28263158e+01,  0.00000000e+00,
         7.52727273e+01,  9.24074074e-01,  1.50000000e+00,
         7.61428571e+00,  5.00000000e-01,  8.66025404e-01],
       [ 3.48596491e+01,  1.67210526e+01,  0.00000000e+00,
         6.81818182e+01,  1.36296296e+00,  9.00000000e-01,
         7.85428571e+00,  8.66025404e-01,  5.00000000e-01],
       [ 3.64543860e+01,  2.08877193e+01,  0.00000000e+00,
         7.24909091e+01,  1.80740741e+00,  2.50000000e+00,
         7.17142857e+00,  1.00000000e+00,  6.12323400e-17],
       [ 3.60894737e+01,  2.30105263e+01,  2.75000000e+02,
         8.00545455e+01,  1.59444444e+00,  3.80000000e+00,
         6.63428571e+00,  8.66025404e-01, -5.00000000e-01],
       [ 3.56315789e+01,  2.49421053e+01,  9.44000000e+02,
         8.50000000e+01,  1.54259259e+00,  6.50000000e+00,
         4.89428571e+00,  5.00000000e-01, -8.66025404e-01],
       [ 3.46929825e+01,  2.58754386e+01,  6.02000000e+02,
         8.60727273e+01,  1.47407407e+00,  6.200000

In [13]:
X_seq[20147,:,:]

array([[ 3.00000000e+01,  1.58000000e+01,  0.00000000e+00,
         7.00000000e+01,  1.80000000e+00,  2.00000000e-01,
         9.00000000e+00, -5.00000000e-01,  8.66025404e-01],
       [ 2.95000000e+01,  1.41000000e+01,  0.00000000e+00,
         6.50000000e+01,  2.10000000e+00,  3.00000000e-01,
         8.50000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 3.30000000e+01,  1.65000000e+01,  0.00000000e+00,
         6.30000000e+01,  3.70000000e+00,  1.00000000e+00,
         8.60000000e+00,  5.00000000e-01,  8.66025404e-01],
       [ 3.41000000e+01,  2.06000000e+01,  0.00000000e+00,
         7.50000000e+01,  4.10000000e+00,  7.00000000e-01,
         9.50000000e+00,  8.66025404e-01,  5.00000000e-01],
       [ 3.31000000e+01,  2.42000000e+01,  3.40000000e+01,
         7.70000000e+01,  4.40000000e+00,  1.70000000e+00,
         9.10000000e+00,  1.00000000e+00,  6.12323400e-17],
       [ 3.42000000e+01,  2.48000000e+01,  7.53000000e+02,
         8.50000000e+01,  3.70000000e+00,  5.500000

In [21]:
for st, g in df.groupby("Station_Number"):
        print(g)

      Station_Names  Year  Month   Max_Temp   Min_Temp  Rainfall  \
15756       Rangpur  1954      1  26.589474  10.575439      20.0   
15757       Rangpur  1954      2  29.722807  12.826316       0.0   
15758       Rangpur  1954      3  34.859649  16.721053       0.0   
15759       Rangpur  1954      4  36.454386  20.887719       0.0   
15760       Rangpur  1954      5  36.089474  23.010526     275.0   
...             ...   ...    ...        ...        ...       ...   
16471       Rangpur  2013      8  36.000000  26.200000     268.0   
16472       Rangpur  2013      9  35.500000  26.100000     229.0   
16473       Rangpur  2013     10  33.400000  23.200000     280.0   
16474       Rangpur  2013     11  30.400000  16.300000       0.0   
16475       Rangpur  2013     12  28.700000  13.700000       0.0   

       Relative_Humidity  Wind_Speed  Cloud_Coverage  Bright_Sunshine  \
15756          81.527273    0.753704             1.3         6.282857   
15757          75.272727    0.924074 

In [22]:
gs = df.groupby("Station_Number")

In [27]:
gs['Relative_Humidity'].to_numpy()

AttributeError: 'SeriesGroupBy' object has no attribute 'to_numpy'